In [ ]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

In [ ]:
from model import Model

baseline_structure =[ 
        {'in_channels':3, 'out_channels':32, 'stride1':2, 'kernel_size':3,  'padding':1, 'conv':'real', 'act_fn':'sign', 'dropout':0,},
        {'in_channels':32, 'out_channels':64, 'stride1':1,'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':64, 'out_channels':128, 'stride1':2,'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':128, 'out_channels':128, 'stride1':1,'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':128, 'out_channels':256, 'stride1':2, 'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':256, 'out_channels':256, 'stride1':1, 'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':256, 'out_channels':512, 'stride1':2, 'kernel_size':3,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':512, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':512, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':512, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':512, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':512, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':512, 'out_channels':1024, 'stride1':2, 'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0},
        {'in_channels':1024, 'out_channels':1024, 'stride1':1,'kernel_size':1,  'padding':0, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0.5},
        {'in_channels':1024, 'out_channels':10, 'stride1':1, 'kernel_size':1,  'padding':0, 'conv':'real', 'act_fn':'sign', 'dropout':0.5},]

model = Model(structure=baseline_structure, 
              adam_init_lr=0.01, 
              lr_patience=50,
              limit_conv_weight=True,
              limit_bn_weight=True)
print(model)
print(model.hparams)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(model, datamodule=cifar10_dm)

In [ ]:
trainer.test(model, datamodule=cifar10_dm)
print(checkpoint_callback.best_model_score)
print(checkpoint_callback.best_model_path)